# Fine Grained Authorization for Retrieval Augmented Generation (RAG)

In the previous step we installed and launched an instance of SpiceDB. Now let's start building our RAG pipeline with fine grained authorization. 

### Part 1: Add Secrets & Setup

Download the `requirements.txt` file from this directory and run the following command to install all dependencies for the workshop. 

In [ ]:
%pip install -r requirements.txt

This walkthrough requires the following environment variables. Create a file named `.env` in your working directory and add these variables:

- ```OPENAI_API_KEY=<add OpenAI key>```
- ```PINECONE_API_KEY=<add Pinecone key>```
- ```SPICEDB_API_KEY=rag-rebac-walkthrough```
- ```SPICEDB_ADDR=localhost:50051```

Load the secrets into your app from the .env file

In [ ]:
from dotenv import load_dotenv

#load secrets from .env file
load_dotenv()

### Part 2: Write a schema to SpiceDB

In today's scenario, we will be authorizing access to view blog articles.

We will begin by defining the authorization logic for our example. To do this, we write a [schema](https://authzed.com/docs/spicedb/concepts/schema) to SpiceDB. The schema below defines two object types, ```user``` and ```article```. Users can relate to a document as a ```viewer``` and any user who is related to a document as a ```viewer``` can ```view``` the document.

In [ ]:
from authzed.api.v1 import (
    Client,
    WriteSchemaRequest,
)

import os

#change to bearer_token_credentials if you are using tls
from grpcutil import insecure_bearer_token_credentials

SCHEMA = """definition user {}

definition article {
    relation viewer: user

    permission view = viewer
}"""

client = Client(os.getenv('SPICEDB_ADDR'), insecure_bearer_token_credentials(os.getenv('SPICEDB_API_KEY')))

try:
    resp = await(client.WriteSchema(WriteSchemaRequest(schema=SCHEMA)))
except Exception as e:
    print(f"Write schema error: {type(e).__name__}: {e}")

### Part 3: Write a Relationship to SpiceDB

Now, we write relationships to SpiceDB that specify that Tim is a viewer of document 123 and 456.

After these relationships are written, any permission checks to SpiceDB will reflect that Tim can view documents 123 and 456.

In [ ]:
from authzed.api.v1 import (
    ObjectReference,
    Relationship,
    RelationshipUpdate,
    SubjectReference,
    WriteRelationshipsRequest,
)

try:
    resp = await (client.WriteRelationships(
        WriteRelationshipsRequest(
            updates=[
                RelationshipUpdate(
                    operation=RelationshipUpdate.Operation.OPERATION_TOUCH,
                    relationship=Relationship(
                        resource=ObjectReference(object_type="article", object_id="123"),
                        relation="viewer",
                        subject=SubjectReference(
                            object=ObjectReference(
                                object_type="user",
                                object_id="tim",
                            )
                        ),
                    ),
                ),
                RelationshipUpdate(
                    operation=RelationshipUpdate.Operation.OPERATION_TOUCH,
                    relationship=Relationship(
                        resource=ObjectReference(object_type="article", object_id="456"),
                        relation="viewer",
                        subject=SubjectReference(
                            object=ObjectReference(
                                object_type="user",
                                object_id="tim",
                            )
                        ),
                    ),
                ),
            ]
        )
    ))
except Exception as e:
    print(f"Write relationships error: {type(e).__name__}: {e}")

### Part 4: Simulate a real-world RAG scenario

We now define a Pinecone serverless index.

Pinecone is a specialized database designed for handling vector-based data. Their serverless product makes it easy to get started with a vector DB.

In [ ]:
#from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from pinecone import Pinecone
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "oscars"

pc.create_index(
    name=index_name,
    dimension=1024,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

We are simulating a real-world RAG (retrieval-augmented generation) scenario by embedding a completely fictional string: "Bill Gates won the 2025 Oscar for best football movie.". Since LLMs don't "know" this fact (as it's made up), we mimic a typical RAG case where private or unknown data augments prompts.

In this example, we also specify metadata like article_id to track which article the string comes from. The article_id is important for linking embeddings to objects that users are authorized on.

In [ ]:
from langchain_pinecone import PineconeEmbeddings
from langchain_pinecone import PineconeVectorStore

from langchain.schema import Document
import os

# Create an object that has the contents of the articles and specifies the document_id as metadata.

documents = [
    Document(
        page_content="Bill Gates won the 2025 Oscar for best football movie",
        metadata={"article_id": "123"}
    ),
    Document(
        page_content="The revenue for Q4 2025 is one billion dollars!",
        metadata={"article_id": "456"}
    )
]


# Initialize LangChain embeddings
embeddings = PineconeEmbeddings(
    model="multilingual-e5-large",
    pinecone_api_key=os.environ.get("PINECONE_API_KEY")
)

# Create vector store and upsert both documents
docsearch = PineconeVectorStore.from_documents(
    documents=documents,
    index_name="oscars",
    embedding=embeddings,
    namespace="oscar"
)

### Part 5: Make a request when the user is authorized to view the necessary contextual data

Next, we will query SpiceDB for a list of documents that Tim is allowed to view. Here, we use the [LookupResources API](https://buf.build/authzed/api/docs/main:authzed.api.v1#authzed.api.v1.LookupResourcesRequest) to obtain a list of articles that ```tim``` has ```view``` permission on. 

In [ ]:
from authzed.api.v1 import (
    LookupResourcesRequest,
    ObjectReference,
    SubjectReference,
)

subject = SubjectReference(
    object=ObjectReference(
        object_type="user",
        object_id="tim",
    )
)

def lookupArticles():
    return client.LookupResources(
        LookupResourcesRequest(
            subject=subject,
            permission="view",
            resource_object_type="article",
        )
    )
try:
    resp = lookupArticles()

    authorized_articles = []

    async for response in resp:
            authorized_articles.append(response.resource_object_id)
except Exception as e:
    print(f"Lookup error: {type(e).__name__}: {e}")

print("Article IDs that Tim is authorized to view:")
print(authorized_articles)

We can now issue a prompt to GPT-3.5, enhanced with relevant data that the user is authorized to access. This ensures that the response is based on information the user is permitted to view.

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

# Define the ask function
def ask():
    # Initialize a LangChain object for an OpenAI chat model.
    llm = ChatOpenAI(
        openai_api_key=os.environ.get("OPENAI_API_KEY"),
        model_name="gpt-3.5-turbo",
        temperature=0.0
    )

    # Initialize a LangChain object for a Pinecone index with an OpenAI embeddings model.
    knowledge = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        namespace=namespace_name,
        embedding=OpenAIEmbeddings(
            openai_api_key=os.environ["OPENAI_API_KEY"],
            dimensions=1024,
            model="text-embedding-3-large"
        )
    )

    # Initialize a retriever with a filter that restricts the search to authorized documents.
    retriever=knowledge.as_retriever(
            search_kwargs={
            "filter": {
                "article_id":
                    {"$in": authorized_articles},
            },
        }
    )

    # Initialize a string prompt template that let's us add context and a question.
    prompt = ChatPromptTemplate.from_template("""Answer the question below using the context:

    Context: {context}

    Question: {question}

    Answer: """)

    retrieval =  RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )

    chain = retrieval | prompt | llm | StrOutputParser()

    question = """Who won the 2025 Oscar for best football movie?"""

    print("Prompt: \n")
    print(question)
    print(chain.invoke(question))

#invoke the ask function
ask()

You can also generate a summary of all the articles that Tim is authorized to view.

In [ ]:
#Summarize only articles that the user is authorized to view

async def summarize_accessible_articles(user_id: str):
    from authzed.api.v1 import LookupResourcesRequest, ObjectReference, SubjectReference
    from langchain_pinecone import PineconeVectorStore
    from langchain_openai import OpenAIEmbeddings
    from openai import AsyncOpenAI
    import os

    # Lookup articles
    subject = SubjectReference(
        object=ObjectReference(object_type="user", object_id=user_id)
    )
    response = client.LookupResources(
        LookupResourcesRequest(
            subject=subject,
            permission="view",
            resource_object_type="article",
        )
    )
    authorized_articles = [res.resource_object_id async for res in response]
    print(f"🔍 {user_id} can view articles: {authorized_articles}")

    if not authorized_articles:
        return "❌ No accessible articles."

    # Setup LangChain retriever w/ filter
    knowledge = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        namespace=namespace_name,
        embedding=OpenAIEmbeddings(
            openai_api_key=os.environ["OPENAI_API_KEY"],
            dimensions=1024,
            model="text-embedding-3-large"
        )
    )

    retriever = knowledge.as_retriever(
        search_kwargs={
            "filter": {"article_id": {"$in": authorized_articles}},
            "k": 100  # Ensure we get all matches
        }
    )

    docs = await retriever.ainvoke("Give me all the contents to summarize")

    if not docs:
        return "❌ No content found."

    combined_text = "\n\n".join([d.page_content for d in docs])

    # 3️⃣ Summarize using OpenAI
    summary_prompt = (
        "You are an AI assistant. Based ONLY on the following articles, "
        "generate a concise summary of their contents. Do not use any outside knowledge.\n\n"
        + combined_text
        + "\n\nSummary:"
    )

    openai_client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    chat_response = await openai_client.chat.completions.create(
        messages=[{"role": "user", "content": summary_prompt}],
        model="gpt-4",
        temperature=0
    )

    return chat_response.choices[0].message.content

In [ ]:
summary = await summarize_accessible_articles("tim")
print("📄 Summary of accessible articles:")
print(summary)

### Part 6: Make a request when the user is NOT authorized to view the necessary contextual data

Now, let's see what happens when Tim is not authorized to view the document.

First, we will delete the relationship that related Tim as a viewer to document 123.

In [ ]:
try: 
    resp = await client.WriteRelationships(
        WriteRelationshipsRequest(
            updates=[
                RelationshipUpdate(
                    operation=RelationshipUpdate.Operation.OPERATION_DELETE,
                    relationship=Relationship(
                        resource=ObjectReference(object_type="article", object_id="123"),
                        relation="viewer",
                        subject=SubjectReference(
                            object=ObjectReference(
                                object_type="user",
                                object_id="tim",
                            )
                        ),
                    ),
                ),
            ]
        )
    )
except Exception as e:
    print(f"Write relationships error: {type(e).__name__}: {e}")

Next, we will update the list of documents that Tim is authorized to view.

In [ ]:
#this function was defined above
try:
        resp = lookupArticles()

        authorized_articles = []

        async for response in resp:
                authorized_articles.append(response.resource_object_id)
except Exception as e:
    print(f"Lookup error: {type(e).__name__}: {e}")

print("Documents that Tim can view:")
print(authorized_articles)

Now, we can run our query again. 

Note that we no longer recieve a completion that answers our question because Tim is no longer authorized to view the document that contains the context required to answer the question.

In [ ]:
#this function was defined above
ask()

### Part 7: Conclusion and Next Steps 

You can now delete your Pinceone index if you'd like to.

In [ ]:
pc.delete_index(index_name)